In [1]:
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
from netCDF4 import Dataset as NetCDFFile
import numpy as N
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy.ma as ma
from scipy.interpolate import griddata
from pyresample import geometry,image, kd_tree


In [32]:
# Read in the other products
#nclu=NetCDFFile('rcp45_9100_oxid.nc','r')
nclu=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/hist_01_10.nc','r')
ncvar_net = nclu.variables['ch4_flux'][:]
ncvar_oxid = nclu.variables['ch4_oxid'][:]
ncvar_prod = nclu.variables['ch4_prod'][:]
ncvar_ebul = nclu.variables['ch4_ebul'][:]
ncvar_aere = nclu.variables['ch4_aere'][:]
ncvar_diff = ncvar_net - ncvar_ebul - ncvar_aere
ncvar_diff[ncvar_diff<-0.5]= 0.1
latnc = nclu.variables['lat'][:]
# znc = nclu.variables['level'][:]
lonnc = nclu.variables['lon'][:]
# timenc = nclu.variables['time'][:]
lonnc = ((lonnc - 180) % 360) - 180

nclu_fw=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_01_10.nc','r')
#nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
fw = nclu_fw.variables['FW'][:]

nclu2=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/rcp45_91_00.nc','r')
ncvar2_net = nclu2.variables['ch4_flux'][:]
ncvar2_oxid = nclu2.variables['ch4_oxid'][:]
ncvar2_prod = nclu2.variables['ch4_prod'][:]
ncvar2_ebul = nclu2.variables['ch4_ebul'][:]
ncvar2_aere = nclu2.variables['ch4_aere'][:]
ncvar2_diff = ncvar2_net - ncvar2_ebul - ncvar2_aere
ncvar2_diff[ncvar2_diff<-0.5]= 0.1
nclu_fw2=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp45.nc','r')
#nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
fw2 = nclu_fw2.variables['FW'][:]

nclu3=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/rcp85_91_00.nc','r')
ncvar3_net = nclu3.variables['ch4_flux'][:]
ncvar3_oxid = nclu3.variables['ch4_oxid'][:]
ncvar3_prod = nclu3.variables['ch4_prod'][:]
ncvar3_ebul = nclu3.variables['ch4_ebul'][:]
ncvar3_aere = nclu3.variables['ch4_aere'][:]
ncvar3_diff = ncvar3_net - ncvar3_ebul - ncvar3_aere
ncvar3_diff[ncvar3_diff<-0.5]= 0.1
nclu_fw3=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp85.nc','r')
#nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
fw3 = nclu_fw3.variables['FW'][:]


In [34]:
#http://matplotlib.org/basemap/users/mapsetup.html

fig = plt.figure(figsize=(15,25))

#ax1 = fig.add_subplot(221)
ax1 = fig.add_subplot(311)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) ",fontsize=20)
#ax1.set_title("$CH_4$ oxidation flux (gC-$CH_4$/$m^2$/yr) ",fontsize=20)
#ax1.set_title("$CH_4$ production (gC-$CH_4$/$m^2$/yr)",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr)",fontsize=20)
#ax1.set_title("$CH_4$ Ebullition (gC-$CH_4$/$m^2$/yr)",fontsize=20)
ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr)",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
#map = Basemap(projection='cyl', llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=-180,urcrnrlon=180,resolution='c')
#map = Basemap(projection='merc', llcrnrlat=-80,urcrnrlat=80,\
#            llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = N.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
# clevs = N.arange(0,7,0.5)
clevs = N.power(10.0, N.arange(-1.0,2.5,0.5))

#ncvar_maize[0,0,:,:][N.isnan(ncvar_maize[0,0,:,:])] = -9999
#maize[0,1,:,:][N.isnan(maize[0,1,:,:])] = -9999
# ncvar_maize[:,:] = ma.masked_where(ncvar_maize[:,:]<0,ncvar_maize[:,:])
# ncvar_maize[ncvar_maize>-1 and ncvar_maize < 0] = 0.01\
ncvar_net = N.clip(ncvar_net, 0.0, 100)
#ncvar_maize[0,0,:,:] = ma.masked_where(ncvar_maize[0,0,:,:]<=0,ncvar_maize[0,0,:,:])
#ncvar_maize[0,0,:,:] = ma.masked_where(maize[0,1,:,:]<=0,ncvar_maize[0,0,:,:])
##mdata = maskoceans(lon, lat, ncvar_net[:,:])

# cs1 = map.contourf(x,y,mdata,clevs,cmap=plt.cm.jet)
# cs1 = map.contourf(x,y,ncvar_maize[:,:], [100], colors="black")
cs1 = map.pcolormesh(x,y,ncvar_diff[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100),cmap=plt.cm.jet) # use log color scale,                   
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
#plt.title('Maize harvested area fraction')
cbar.ax.tick_params(labelsize=18) 


#ax2 = fig.add_subplot(222)
#ax2.set_title("Soil $CH_4$ oxidation (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
##map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
##        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
#map = Basemap(projection='cyl', llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=0,urcrnrlon=360,resolution='c')
## draw coastlines, state and country boundaries, edge of map.
##map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
#clevs = N.arange(0,7,0.5)
## clevs = N.power(10.0, N.arange(-1.0,2.5,0.5))
##clevs = [0,1.5,3,4.5,6,7.5,9,10.5,12,13.5,15]
## cs = map.contourf(x,y,maize[:,:],clevs,cmap=plt.cm.jet)

##maize[:,:] = ma.masked_where(maize[:,:]==0,ncvar_maize[:,:])

#cs = map.pcolormesh(x,y,maize[:,:],
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
#                                                      # omit this to use linear
#                                                     # color scale
#cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
#cbar.ax.tick_params(labelsize=18) 


ax1 = fig.add_subplot(312)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ oxidation (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ production (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ ebullition (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
#map = Basemap(projection='cyl', llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=-180,urcrnrlon=180,resolution='c')
#map = Basemap(projection='merc', llcrnrlat=-80,urcrnrlat=80,\
#            llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = N.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
# Transfer wetchat data to gCH4/m2/yr
# ncvar_maize2 = ncvar_maize2 * 365.0 / 1000.0
#clevs = N.arange(0,0.4,0.05)
#clevs = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
# cs = map.contourf(x,y,(fw_max[0,:,:]),clevs,cmap=plt.cm.jet)
ncvar2_net = N.clip(ncvar2_net, 0., 100)
#cs = map.pcolormesh(x,y,ncvar_maize2[0,0,:,:],
cs = map.pcolormesh(x,y,ncvar2_diff[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100),cmap=plt.cm.jet) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=18) 
#cbar = map.colorbar(cs,location='bottom',pad="5%")
#cbar.ax.tick_params(labelsize=18) 

ax1 = fig.add_subplot(313)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ oxidation (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ production (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ ebullition (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
#map = Basemap(projection='cyl', llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=-180,urcrnrlon=180,resolution='c')
#map = Basemap(projection='merc', llcrnrlat=-80,urcrnrlat=80,\
#            llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = N.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
# Transfer wetchat data to gCH4/m2/yr
# ncvar_maize2 = ncvar_maize2 * 365.0 / 1000.0
#clevs = N.arange(0,0.4,0.05)
#clevs = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
# cs = map.contourf(x,y,(fw_max[0,:,:]),clevs,cmap=plt.cm.jet)
ncvar3_net = N.clip(ncvar3_net, 0., 100)
#cs = map.pcolormesh(x,y,ncvar_maize2[0,0,:,:],
cs = map.pcolormesh(x,y,ncvar3_diff[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100),cmap=plt.cm.jet) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=18) 
#cbar = map.colorbar(cs,location='bottom',pad="5%")
#cbar.ax.tick_params(labelsize=18) 


#ax1 = fig.add_subplot(224)
#ax1.set_title("Soil $CH_4$ oxidation (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
##map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
##        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
#map = Basemap(projection='cyl', llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=0,urcrnrlon=360,resolution='c')
## draw coastlines, state and country boundaries, edge of map.
#map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
##lon,lat = N.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
##x,y = map(lon,lat)
##clevs = N.arange(0,0.4,0.05)
##clevs = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
##cs = map.contourf(x,y,(maize2[:,:]),clevs,cmap=plt.cm.jet)
##maize2[:,:] = ma.masked_where(maize2[:,:]==0,ncvar_maize2[:,:])
#cs = map.pcolormesh(x,y,maize2[:,:],
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
#                                                      # omit this to use linear
#                                                     # color scale
#cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
#cbar.ax.tick_params(labelsize=18) 

#plt.show()
plt.savefig('Diff_CH4.jpg',dpi=300)

In [ ]:
lonnc2

In [ ]:
lonnc2 % 360
